In [1]:
import re
import pandas as pd
import numpy as np
import dataframe
from bs4 import BeautifulSoup
import socket 
import time
import requests
from selenium import webdriver
from urllib.parse import quote_plus
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import urllib.request
import urllib.parse

In [2]:
ceo_df = pd.read_csv(r'C:\Users\Jeeho\Documents\Python Scripts\Capstone project\Web_Scrap\fg500_CEOs_v4_all(Apr_10_2020).csv')

In [3]:
c_sample = ceo_df['CEO']

In [4]:
browser = webdriver.Chrome(r"C:\Users\Jeeho\AppData\Local\Programs\Python\Python36\chromedriver.exe")


ceo_list = c_sample
values =[]
df = pd.DataFrame(columns = ['ceo'])

for ceo in ceo_list:
    baseUrl = 'https://www.google.com/search?q='
    browser.get(baseUrl + ceo)
    table = browser.find_elements_by_css_selector('div.ifM9O') 

    for row in table:
        ceo = str(([c.text for c in row.find_elements_by_css_selector('div.kno-ecr-pt.PZPZlf.gsmt.i8lZMc')])).strip('[]').strip("''")
    for row in table:
        value = str(([c.text for c in row.find_elements_by_css_selector('div.Z1hOCe')])).strip('[]').strip("''")
    if not table:
        value = 'Null'
    
    values.append(value)
    
    s = pd.Series(ceo,index=df.columns)
    vs = pd.Series(values) 

    df = df.append(s,ignore_index=True)

    time.sleep(2) # To make the scraping slower
    
    
df['value'] = vs  
print(df)

browser.close()

                                  ceo  \
0                       Doug McMillon   
1                        Dai Houliang   
2                     Ben van Beurden   
3                                       
4                                       
5                      Amin H. Nasser   
6                          Bob Dudley   
7                        Darren Woods   
8                       Herbert Diess   
9                         Akio Toyoda   
10                           Tim Cook   
11                     Warren Buffett   
12                         Jeff Bezos   
13                                      
14                                      
15                    Ivan Glasenberg   
16                     Brian S. Tyler   
17                      Ola Källenius   
18                     Larry J. Merlo   
19                   Patrick Pouyanné   
20                                      
21                        Jeremy Weir   
22                          Terry Gou   
23              

In [5]:
#Make a copy of the original scraped dataset
df1 = df

In [6]:
#reindex the data
re_index = pd.Series(range(1,500))  
df1 = df1.set_index([re_index])

Rename the unnamed column as index

In [8]:
#ceo list from 1-266
df_1 = df1.iloc[0:266]

#ceo list from 267 to 500
df_2 = df1.iloc[266:499]

#reindex the df_2
re_index2 = pd.Series(range(268,501))  
df_2 = df_2.set_index([re_index2])

#Merge the reindexed lists
df1=df_1.append(df_2, ignore_index=False)


In [10]:
# convert the index to a column and name as company_rank 
df1['index1'] = df1.index

df1 = df1.rename(columns={'index1': 'company_rank'})

In [12]:
#reindex the data as starting from 1 to 499
re_index = pd.Series(range(1,500))  
df1 = df1.set_index([re_index])

Clear rows. Select the rows having values only

In [20]:
#Select only the rows with values to proceed RegEx later
# exclude the rows with ceo value having ''
df_n = df1[df1['ceo'] != '']

#exlucde the rows if the value of 'value' column is null
df_nv = df_n[(df_n['value'] != 'Null')]

Splite the 'value' column using regex

Succeed to split the values. However, only some of the should be selected: Born, Nationality, and Education

In [22]:
# For Born
df_nv['Born'] = df_nv['value'].str.extract(r"Born: (.+?)\'\," )

# For Nationality 
df_nv['Nationality'] = df_nv['value'].str.extract(r"Nationality: (.+?)\'\," )


# For Education
## Select the strings end with "', '" and the strings end with nothing
Education = df_nv['value'].str.extract(r"Education: (.+?)\'\,\s\'|Education: (.+)?|Alma mater: (.+?)\'\,\s\'")

## merge the extracted two columes
df_nv['Education'] = pd.concat([Education[0].dropna(), Education[1].dropna(), Education[2].dropna()]).reindex_like(Education)

#See the full contents of the rows of value
pd.set_option('display.max_colwidth', -1)


C:\Users\Jeeho\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Jeeho\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\Jeeho\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html

Extract only Birth of Year only as BoY

In [23]:
df_nv['BoY'] = df_nv['Born'].str.extract(pat = "([0-9]{4})" )

C:\Users\Jeeho\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Extract Nationality strings from Born based on 1) the postal abbreviations of the US and 2) the country names

In [24]:
# Create a list of the abbr

us_postal_abbr = {'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 
                  'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 
                  'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 
                  'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 
                  'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WY'}

reorder the index to run the for loop

In [25]:
df_nv_index = pd.Series(range(len(df_nv))) 

In [26]:
df_nv = df_nv.set_index([df_nv_index])

In [27]:
# Create a column for the extracted us abbreviation code 
df_nv['us_abbr'] = df_nv['Born'].str.extract(pat = "([A-Z]{2})")

# Insert value 'American' in the 'Nationality' column if where the us_abbr matchs with the list of abbr
df_nv.loc[df_nv['us_abbr'].isin(us_postal_abbr), 'Nationality'] = 'American'

For nationalities

In [28]:
pd.set_option('display.max_rows', None)

Catch the missing countries among the values

In [29]:
#Extract the United States and United Kingdom and the other country names
bn = df_nv['Born'].str.extract(".*?([A-Za-z]{6}\s[A-Za-z]{6,7})$|.*?([A-Za-z]{5,11})$")

## merge the extracted two columes
df_nv['born_country'] = pd.concat([bn[0].dropna(), bn[1].dropna()]).reindex_like(bn)

#Replacing the NaN values of Nationality column with newly extracted values 
df_nv['Nationality'].fillna(df_nv['born_country'], inplace=True)

In [30]:
#drop value, us_abbr, and born_coutry columns
df_nv = df_nv.drop(['value', 'us_abbr', 'born_country'], axis = 1)


In [32]:
#reindex the data as starting from 1
re_index_nv = pd.Series(range(1,245))  
df_nv = df_nv.set_index([re_index_nv])

In [34]:
df_nv.to_csv(r'D:\Yeshiva_Univ\Courses\Capstone_project\CEOs_Culture_project\Data\ceo_information\CEOs_info_v6.1(Apr_10_2020).csv')